<a href="https://colab.research.google.com/github/Tomawock/MLDM_COVID-19/blob/master/datasets/dataset_finale_medie_ponderate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creazione di vari Dataset dove son già raggruppate le X e le Y in base alle necessità

In questo dataset le medie sono ponderate (con maggiore importanza per le date più a ritroso nel tempo). Il resto ricalca ciò che avviene in "dataset_finale_medie.ipynb".

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import os
import random
os.chdir('/content/drive/Shared drives/MLDM progetto/Datasets/dataset_finali')

In [3]:
#Carica dati dai file
dataset_completo = pd.read_csv("dataset_completo.csv")

In [137]:
def media_ponderata(giorni_selezionati, index_oldest):
  ponderatore = giorni_selezionati.index-index_oldest+1
  return giorni_selezionati.multiply(pd.Series(ponderatore, index=giorni_selezionati.index), axis="index").sum()/sum(ponderatore)


##Creazione dataset denominato: dataset_finale_medie_ponderate.csv

In questo dataset le X sono composte da tutti i parametri presi sei giorni stocasticamente compresi fra le due settimane precedenti. Di questi dati viene computata la media aritmetica.


In [140]:
random.seed(1)

col=list(dataset_completo.columns)
col.remove("Data")

for i in range(0,dataset_completo.shape[0]-18):
  data_previsione=dataset_completo.loc[i+14, "Data"]
  giorni_precedenti=dataset_completo.loc[i:i+14, :]
  giorni_selezionati=random.sample(range(i,i+14), 6)
  giorni_estratti=dataset_completo.loc[giorni_selezionati, dataset_completo.columns != "Data"]
  media_passato=media_ponderata(giorni_estratti, i).to_frame()
  
  media_passato=media_passato.pivot_table(columns=col).add_prefix("pass_")
  media_passato["Data"]=data_previsione
  fut1=pd.to_numeric(dataset_completo.loc[i+16, dataset_completo.columns != "Data"]).to_frame().pivot_table(columns=col).add_prefix("fut1_").reset_index()
  fut2=pd.to_numeric(dataset_completo.loc[i+18, dataset_completo.columns != "Data"]).to_frame().pivot_table(columns=col).add_prefix("fut2_").reset_index()
  riga=pd.concat([media_passato, fut1, fut2], axis=1).drop(columns="index")
  if i==0:
    finale=pd.DataFrame(columns=riga.columns)
  
  finale=pd.concat([finale, riga], axis=0, ignore_index=True)

finale

,pass_Altezza Neve,pass_Ammoniaca,pass_Benzene,pass_Biossido di Azoto,pass_Biossido di Zolfo,pass_BlackCarbon,pass_Direzione Vento,pass_Livello Idrometrico,pass_Monossido di Azoto,pass_Monossido di Carbonio,pass_Ossidi di Azoto,pass_Ozono,pass_PM10 (SM2005),pass_Particelle sospese PM2.5,pass_Precipitazione,pass_Radiazione Globale,pass_Temperatura,pass_Umidità Relativa,pass_Velocità Vento,pass_deceduti,pass_dimessi_guariti,pass_isolamento_domiciliare,pass_nuovi_positivi,pass_ricoverati_con_sintomi,pass_tamponi,pass_terapia_intensiva,pass_totale_casi,pass_totale_ospedalizzati,pass_totale_positivi,pass_variazione_totale_positivi,Data,fut1_Altezza Neve,fut1_Ammoniaca,fut1_Benzene,fut1_Biossido di Azoto,fut1_Biossido di Zolfo,fut1_BlackCarbon,fut1_Direzione Vento,fut1_Livello Idrometrico,fut1_Monossido di Azoto,...,fut1_dimessi_guariti,fut1_isolamento_domiciliare,fut1_nuovi_positivi,fut1_ricoverati_con_sintomi,fut1_tamponi,fut1_terapia_intensiva,fut1_totale_casi,fut1_totale_ospedalizzati,fut1_totale_positivi,fut1_variazione_totale_positivi,fut2_Altezza Neve,fut2_Ammoniaca,fut2_Benzene,fut2_Biossido di Azoto,fut2_Biossido di Zolfo,fut2_BlackCarbon,fut2_Direzione Vento,fut2_Livello Idrometrico,fut2_Monossido di Azoto,fut2_Monossido di Carbonio,fut2_Ossidi di Azoto,fut2_Ozono,fut2_PM10 (SM2005),fut2_Particelle sospese PM2.5,fut2_Precipitazione,fut2_Radiazione Globale,fut2_Temperatura,fut2_Umidità Relativa,fut2_Velocità Vento,fut2_deceduti,fut2_dimessi_guariti,fut2_isolamento_domiciliare,fut2_nuovi_positivi,fut2_ricoverati_con_sintomi,fut2_tamponi,fut2_terapia_intensiva,fut2_totale_casi,fut2_totale_ospedalizzati,fut2_totale_positivi,fut2_variazione_totale_positivi
0,116.370376,12.503056,0.784844,29.580494,2.425786,1.885000,188.962778,35.591691,0.447083,0.610197,46.729839,38.991840,23.957298,18.871340,0.056944,123.995097,4.334077,71.866113,2.346755,69.600000,236.000000,261.475000,258.050000,840.700000,9.716275e+03,179.350000,1587.125000,1020.050000,1281.525000,170.275000,2020-03-09,116.508564,13.275000,0.680952,31.763360,2.866919,1.5,190.502696,28.951725,0.754167,...,900.0,1351.0,1489.0,3852.0,25629.0,560.0,7280.0,4412.0,5763.0,1336.0,109.034233,14.012500,0.533333,20.682788,2.456760,0.7,202.414057,28.240583,0.375000,0.516473,27.530027,49.417757,24.709677,19.966667,0.001532,152.153664,11.042177,74.557500,1.945538,890.0,1198.0,2647.0,1095.0,4435.0,32700.0,650.0,9820.0,5085.0,7732.0,836.0
1,122.023490,12.071930,0.718494,29.356460,2.429167,1.615789,183.248371,36.250057,0.411404,0.610248,42.681410,41.772981,20.920174,17.074091,0.041121,115.848187,4.473883,70.152075,2.123922,182.315789,414.000000,629.947368,638.912281,1680.017544,1.444525e+04,301.035088,3207.315789,1981.052632,2611.000000,529.912281,2020-03-10,110.362902,17.637500,0.745455,30.971685,2.516601,1.8,204.788320,28.628933,0.362500,...,1085.0,2044.0,1445.0,4247.0,29534.0,605.0,8725.0,4852.0,6896.0,1133.0,112.331299,11.088889,0.595652,14.829834,1.991204,1.0,146.477153,32.057624,0.141667,0.494544,19.485288,46.814770,25.650794,20.806452,0.068590,42.026965,6.675531,86.111540,1.919871,966.0,1660.0,3429.0,1865.0,4898.0,37138.0,732.0,11685.0,5630.0,9059.0,1327.0
2,124.164390,11.408401,0.710671,27.759505,2.459146,2.143902,183.704291,38.249879,0.397764,0.600862,43.756249,46.010132,18.859414,15.557392,0.010107,162.730612,5.105086,58.704520,2.315021,144.000000,376.024390,584.048780,553.317073,1395.121951,1.390139e+04,285.000000,2784.195122,1680.121951,2264.170732,462.512195,2020-03-11,109.034233,14.012500,0.533333,20.682788,2.456760,0.7,202.414057,28.240583,0.375000,...,1198.0,2647.0,1095.0,4435.0,32700.0,650.0,9820.0,5085.0,7732.0,836.0,112.449581,14.544444,0.560870,16.264336,1.800396,1.8,166.463890,29.875236,0.233333,0.521769,21.749993,42.753186,18.000000,15.333333,0.001614,127.353711,5.350083,79.784170,1.404195,1218.0,2011.0,3776.0,1587.0,5500.0,40369.0,767.0,13272.0,6267.0,10043.0,984.0
3,118.281517,13.987805,0.749084,30.251011,2.433280,2.024390,170.265320,33.526839,0.384553,0.621435,45.13

In [141]:
colonne_non_ordinate=list(finale.columns)
colonne_non_ordinate.remove("Data")
elenco_ordinato_colonne=["Data"]

elenco_ordinato_colonne=elenco_ordinato_colonne+colonne_non_ordinate
finale=finale.reindex(columns=elenco_ordinato_colonne)
finale.to_csv("dataset_finale_medie_ponderate_rnd1.csv", index=False)